# Bitcoin price forecasting with PySpark
## Big Data Computing final project - A.Y. 2022 - 2023
Prof. Gabriele Tolomei

MSc in Computer Science

La Sapienza, University of Rome

### Author
Corsi Danilo - corsi.1742375@studenti.uniroma1.it



# Global Constants


In [ ]:
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"

GDRIVE_DATASET_RAW_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/raw"
GDRIVE_DATASET_TEMP_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/temp"
GDRIVE_DATASET_OUTPUT_DIR = GDRIVE_DIR + "/MyDrive/BDC/project/datasets/output"

GDRIVE_DATASET_NAME = "bitcoin_blockchain_data_1h"
GDRIVE_DATASET_NAME_TRAIN = GDRIVE_DATASET_NAME + "_train"
GDRIVE_DATASET_NAME_TEST = GDRIVE_DATASET_NAME + "_test"

GDRIVE_DATASET_NAME_EXT_TRAIN  = "/" + GDRIVE_DATASET_NAME_TRAIN + ".parquet"
GDRIVE_DATASET_NAME_EXT_TEST = "/" + GDRIVE_DATASET_NAME_TEST + ".parquet"

GDRIVE_DATASET_TRAIN = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TRAIN
GDRIVE_DATASET_TEST = GDRIVE_DATASET_OUTPUT_DIR + GDRIVE_DATASET_NAME_EXT_TEST

SLOW_OPERATION = False

#  Import Python packages

In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from itertools import cycle

import plotly.express as px

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import gc

# Spark + Google Colab Setup

In [ ]:
!pip install pyspark
# Alternatively, if you want to install a specific version of pyspark:
#!pip install pyspark==3.2.1
!pip install -U -q PyDrive # To use files that are stored in Google Drive directly (e.g., without downloading them from an external URL)
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = JAVA_HOME

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

from pyspark.sql import functions as F

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285397 sha256=79bb77ec48798752b65a8491106d6e45caaf591ad04450d4065b7f790874cd96
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 15 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected packag

In [ ]:
# Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set("spark.kryoserializer.buffer.max", "1G").\
                setAppName("BitcoinPriceForecasting").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Point Colaboratory to our Google Drive

from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# Random Forest

## Model preparation ❗

In [ ]:
# load dataset into pyspark dataframe objects
train_df = spark.read.load(GDRIVE_DATASET_TRAIN,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

test_df = spark.read.load(GDRIVE_DATASET_TEST,
                         format="parquet",
                         sep=",",
                         inferSchema="true",
                         header="true"
                    )

After loading the dataset we need to vectorize all of our features into a single column.

To build and compare performance of our three feature set sizes:
* All features, our baseline
* Relevant features
* RFE-selected features

In [ ]:
# TODO: import feature selected form JSON ❗
all_columns = ['market-cap', 'total-bitcoins', 'trade-volume', 'blocks-size', 'avg-block-size', 'n-transactions-total', 'n-transactions-per-block', 'hash-rate', 'difficulty', 'miners-revenue', 'transaction-fees-usd', 'n-unique-addresses', 'n-transactions', 'estimated-transaction-volume-usd']
rel_columns = ['market-cap', 'estimated-transaction-volume-usd', 'blocks-size', 'n-unique-addresses']
sel_columns = ['total-bitcoins', 'blocks-size', 'avg-block-size', 'n-transactions-per-block', 'miners-revenue', 'n-unique-addresses', 'n-transactions']

dep_var = 'market-price'

We'll start by assembling 3 independent VectorAssemblers, 1 for each feature list. To get the 3 Vectorized RDDs, we apply the transform on the data using each:

In [ ]:
def select_features(dataset, features):
  vectorAssembler = VectorAssembler(
    inputCols = features,
    outputCol = 'features')

  dataset = vectorAssembler.transform(dataset)
  dataset = dataset.select(['timestamp','index', 'features', dep_var])
  return dataset

In [ ]:
from pyspark.ml.feature import VectorAssembler

#All columns featurized
all_train_df = select_features(train_df, all_columns)
all_test_df = select_features(test_df, all_columns)

# #Relevant columns featurized
# rel_train_df = select_features(train_df, rel_columns)
# rel_test_df = select_features(test_df, rel_columns)

# #RFE-selected columns featurized
# sel_train_df = select_features(train_df, sel_columns)
# sel_test_df = select_features(test_df, sel_columns)

Then we call the show method on each of the resulting RDDs yields the following vectorized inputs (X) and targets (y).

In [ ]:
def print_df_info(dataset, type):
  print("The shape of the {:s} dataset is {:d} rows by {:d} columns".format(type, dataset.count(), len(dataset.columns)))
  dataset.show(3)

In [ ]:
if SLOW_OPERATION:
  print_df_info(all_train_df, "all columns train")
  print_df_info(all_test_df, "all columns test")

  # print_df_info(rel_train_df, "relevant columns train")
  # print_df_info(rel_test_df, "relevant columns test")

  # print_df_info(sel_train_df, "selected columns train")
  # print_df_info(sel_test_df, "selected columns test")

## Predictions ❗

In [ ]:
def random_forest(train, test):
  from pyspark.ml.regression import RandomForestRegressor
  from pyspark.ml.feature import StringIndexer, VectorAssembler

  # Crea un oggetto RandomForestRegressor: Puoi impostare i parametri desiderati per il modello Random Forest come numero di alberi (numTrees), profondità massima degli alberi (maxDepth), numero massimo di bin per il partizionamento delle features (maxBins), ecc.
  rf = RandomForestRegressor(
      featuresCol="features",  # Colonna vettoriale delle features
      labelCol="market-price",  # Colonna delle etichette di output
  )

  # Addestra il modello: Utilizza il metodo fit() per addestrare il modello sulla tua dataset di addestramento.
  model = rf.fit(train)

  # Effettua le previsioni: Utilizza il modello addestrato per fare previsioni sul tuo dataset di test o su nuovi dati.
  predictions = model.transform(test)

  return predictions, model

In [ ]:
all_predictions_df, all_model = random_forest(all_train_df, all_test_df)
# rel_predictions_df, rel_model = random_forest(rel_train_df, rel_test_df)
# sel_predictions_df, sel_model = random_forest(sel_train_df, sel_test_df)

if SLOW_OPERATION:
  print_df_info(all_predictions_df, "all columns prediction")
  # print_df_info(rel_predictions_df, "relevant columns prediction")
  # print_df_info(sel_predictions_df, "selected columns prediction")

In [ ]:
def show_results(train, test, pred):
  trace1 = go.Scatter(
      x = train['timestamp'],
      y = train['market-price'].astype(float),
      mode = 'lines',
      name = 'Train'
  )

  trace2 = go.Scatter(
      x = test['timestamp'],
      y = test['market-price'].astype(float),
      mode = 'lines',
      name = 'Test'
  )

  trace3 = go.Scatter(
      x = pred['timestamp'],
      y = pred['prediction'].astype(float),
      mode = 'lines',
      name = 'Prediction'
  )

  layout = dict(
      title='Train, test and prediction set  with Rangeslider',
      xaxis=dict(
          rangeselector=dict(
              buttons=list([
                  #change the count to desired amount of months.
                  dict(count=1,
                      label='1m',
                      step='month',
                      stepmode='backward'),
                  dict(count=6,
                      label='6m',
                      step='month',
                      stepmode='backward'),
                  dict(count=12,
                      label='1y',
                      step='month',
                      stepmode='backward'),
                  dict(count=36,
                      label='3y',
                      step='month',
                      stepmode='backward'),
                  dict(step='all')
              ])
          ),
          rangeslider=dict(
              visible = True
          ),
          type='date'
      )
  )

  data = [trace1, trace2, trace3]
  fig = dict(data=data, layout=layout)
  iplot(fig, filename = "Train, test and prediction set  with Rangeslider")

In [ ]:
show_results(train_df.toPandas(), test_df.toPandas(), all_predictions_df.toPandas())

Output hidden; open in https://colab.research.google.com to view.

## Evaluation ❗

In [ ]:
# Function to evaluate a model
def evaluation(pred, n_features):
  from pyspark.ml.evaluation import RegressionEvaluator

  evaluator = RegressionEvaluator(
      predictionCol="prediction",  # Colonna delle previsioni
      labelCol="market-price",  # Colonna delle etichette di output
  )

  mse = evaluator.evaluate(pred, {evaluator.metricName: "mse"})
  rmse = evaluator.evaluate(pred, {evaluator.metricName: "rmse"})
  r2 = evaluator.evaluate(pred, {evaluator.metricName: "r2"})
  mae = evaluator.evaluate(pred, {evaluator.metricName: "mae"})

  from pyspark.sql.functions import abs, col
  from pyspark.sql import functions as F
  from pyspark.ml.evaluation import RegressionEvaluator

  # Calcola il MAPE
  mape = pred.withColumn("error", abs(col("market-price") - col("prediction")) / col("market-price")) \
          .selectExpr("avg(error) * 100 as mape") \
          .collect()[0]["mape"]

  # adj_r2
  n = pred.count()  # Numero di osservazioni
  p = n_features # Numero di predittori nel modello
  adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))

  print("MSE = %s" % (mse)) # deve essere un valore non negativo, dove un valore di 0 indica una perfetta corrispondenza tra i valori predetti e quelli di riferimento
  print("RMSE = %s" % (rmse)) # dovresti considerare il valore di RMSE in relazione al range dei valori target nel tuo problema specifico
  print("R2 = %s" % (r2)) # piú é vicino ad 1 meglio é
  print("MAE = %s" % (mae)) # può essere utile confrontare il valore di MAE con quello di altri modelli o con il range dei valori target per valutare la sua precisione
  print("MAPE = %s" % (mape)) # di solito viene utilizzato come misura relativa per confrontare la precisione di modelli diversi
  print("ADJ R2 = %s" % (adj_r2)) # piú é vicino ad 1 meglio é

In [ ]:
evaluation(all_predictions_df, len(all_columns))
# evaluation(rel_predictions_df)
# evaluation(sel_predictions_df)

MSE = 77058739.66501749
RMSE = 8778.310752361042
R2 = 0.5875016663735204
MAE = 7484.627081452254
MAPE = 27.13897711289007
ADJ R2 = 0.5872147822865472
